<a href="https://colab.research.google.com/github/chandrasekhar36/Facial_Expression_Recognition/blob/master/train_FER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
from matplotlib import pyplot
from math import sqrt 
import numpy as np 
import scipy.misc 
from IPython.display import display 
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator 
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization,GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.utils import shuffle
from keras.models import load_model
from keras.applications.vgg16 import VGG16
batch_size=64
img_height=48
img_width=48

In [0]:
train_datagen = ImageDataGenerator( 
    rescale=1./255,
    rotation_range = 10,
    horizontal_flip = True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=0.2,
    fill_mode = 'nearest')

test_datagen = ImageDataGenerator( 
    rescale=1./255
    )

In [6]:
training_set = train_datagen.flow_from_directory(
    "/content/drive/My Drive/data_2",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') 

test_set = train_datagen.flow_from_directory(
    "/content/drive/My Drive/data_2",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') 

Found 3317 images belonging to 4 classes.
Found 827 images belonging to 4 classes.


In [7]:
model=VGG16(include_top=False,input_shape=(48,48,3))

58892288/58889256 [==============================] - 1s 0us/step


In [0]:
#flat=Flatten()(model.outputs)
flat = GlobalAveragePooling2D()(model.output)
x = Dense(1024,activation='relu')(flat)
output= Dense(4, activation='softmax')(x)
model = Model(inputs=model.inputs, outputs=output)
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics = ['accuracy'])

In [0]:
modelH=model.fit_generator(training_set,
                         epochs = 200,
                         shuffle=True,
                         validation_data = test_set)

Epoch 1/200
52/52 [==============================] - 388s 7s/step - loss: 1.0078 - accuracy: 0.5761 - val_loss: 1.7879 - val_accuracy: 0.1632
Epoch 2/200
52/52 [==============================] - 396s 8s/step - loss: 0.9358 - accuracy: 0.5957 - val_loss: 1.7319 - val_accuracy: 0.1971
Epoch 3/200
52/52 [==============================] - 377s 7s/step - loss: 0.8984 - accuracy: 0.6180 - val_loss: 1.6672 - val_accuracy: 0.2104
Epoch 4/200
52/52 [==============================] - 375s 7s/step - loss: 0.8391 - accuracy: 0.6497 - val_loss: 1.6643 - val_accuracy: 0.2080
Epoch 5/200
52/52 [==============================] - 375s 7s/step - loss: 0.7790 - accuracy: 0.6816 - val_loss: 1.7292 - val_accuracy: 0.2406
Epoch 6/200
52/52 [==============================] - 381s 7s/step - loss: 0.7586 - accuracy: 0.6883 - val_loss: 2.1080 - val_accuracy: 0.2322
Epoch 7/200
52/52 [==============================] - 379s 7s/step - loss: 0.7272 - accuracy: 0.6994 - val_loss: 2.1627 - val_accuracy: 0.2213
Epoch 

In [0]:
import json
json.dump(str(modelH.history), open('history.csv', 'w'))
model.save('Model.h5')

In [0]:
def plot_confusion_matrix(y_test, y_pred, classes,
                          normalize=False,
                          title='Unnormalized confusion matrix',
                          cmap=plt.cm.Blues):
    cm = confusion_matrix(y_test, y_pred)
    
    if normalize:
        cm = np.round(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], 2)
        
    np.set_printoptions(precision=2)
        
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.min() + (cm.max() - cm.min()) / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True expression')
    plt.xlabel('Predicted expression')
    plt.show()
classes=np.array(('Boredm','Engagement','Confusion','Frustration'))

In [0]:
y_pred_ = modelN.predict(test_set, verbose=1)
y_pred = np.argmax(y_pred_, axis=1)
t_te = np.argmax(y_val, axis=1)
fig = plot_confusion_matrix(y_test=t_te, y_pred=y_pred,
                      classes=classes,
                      normalize=True,
                      cmap=plt.cm.Greys,
                      title='Average accuracy: ' + str(np.sum(y_pred == t_te)/len(t_te)) + ' over test data\n')

In [0]:
y_pred_ = modelN.predict(training_set, verbose=1)
y_pred = np.argmax(y_pred_, axis=1)
t_te = np.argmax(y_val, axis=1)
fig = plot_confusion_matrix(y_test=t_te, y_pred=y_pred,
                      classes=classes,
                      normalize=True,
                      cmap=plt.cm.Greys,
                      title='Average accuracy: ' + str(np.sum(y_pred == t_te)/len(t_te)) + ' over train data\n')